In [2]:
import nltk

In [3]:
import string
import os

In [4]:
import stanfordnlp

from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'C:\Users\spike\python\stanford-corenlp-full-2018-10-05',lang='en')

xnlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos,lemma')

xnlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\spike\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\spike\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\spike\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\spike\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\spike\\stanfordnlp_resources\\en_ewt_models\\en_ewt_to

In [5]:
avoidance=string.punctuation+string.whitespace

In [6]:
def readfile(file):
    list_sents={}
    name=os.path.basename(file)
    if file[-4:]==".txt":
        text=open(file,encoding='utf-8',errors='ignore')
        avoidance=string.punctuation+string.whitespace
        option=("A.","B.","C.","D.","E.","F.","G.","B ","C ","D ","E ","F ","G ")
    #print(avoidance)
        line=''
        for chline in text:
            #for w in option:
                #chline=chline.replace(w," ")           
            for character in chline:
                #if character in option and chline[chline.index(character)+1] in ("."," "):
                    #line+=" "
                if character=="'" or character=='"' or character=="’"or character=="“" or character=="”":
                    line+=character
                elif character in ("_"):
                    line+=" "
                elif character in ("(",")","（","）"):
                    line+=" "+character+' '
                elif character in avoidance:
                    line+=character
                elif ord(character) in range(32,123):# and character not in '\t1234567890':
                    line+=character
                else:
                    line+=""
        
        text=""
    #text=text.join(txt).replace("\n",' ')
        text=line.replace("“",'"').replace("”",'"').replace("’","'").replace("‘","'")

        return text
    else:
        return {}

In [7]:
def lemma(s):
    output=[]
    doc = xnlp(s)
    for sent in doc.sentences:
        for word in sent.words:
            output.append(word.lemma)
    return output


In [8]:
#doc = xnlp("Barack Obama was born in Hawaii.")
#print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [9]:
def gonver(s):
    output=[]
    doc=xnlp(s)
    for word in doc.sentences[0].words:
        output.append(word.governor)
    return output

In [10]:
def dependency(s):
    s=nlp.dependency_parse(s)
    output=[]
    for n in range(1,len(s)+1):
        for w in s:
            if w[2]==n:
                output.append(w)
    return output

In [11]:

#nlp.annotate(text)

In [12]:
def stanz(s): #分析一个句子中的各个成分包括单词、lemma、词性、依存关系、序列
    words=nlp.word_tokenize(s)
    dep=dependency(s)
    tags=[]
    for item in nlp.pos_tag(s):
        tags.append(item[1])
    lemmas=lemma(s)
    gov=[]
    dpg=[]
    govw=[]
    for w in dep:
        gov.append(w[1])
        dpg.append(w[0])
        govw.append(words[w[1]-1])
    length=len(words)+1
    ifhead=[]
    
    for n in range(1,length): 
        if n in gov:
            ifhead.append('head')
        else:
            ifhead.append('nonehead')
        
    tup=zip(words,tags,gov,lemmas,dpg,range(1,length),govw,ifhead)
    output=[]
    for item in tup:
        output.append(item)
    return output

In [13]:
def NT_unit(s): #计算T unit 结构
    n=0
    #print(nlp.parse(s))
    
    parse=nlp.parse(s).split('\r\n')
    print(parse)
    
    for no in range(0,len(parse)):
        #print(line)
        if no!=len(parse) and parse[no][-2:]=='(S' and parse[no+1][-2:]!='(S':
            n+=1
            print(parse[no])
        if parse[no].replace(' ','') in ('(SBAR','(TO'):
            n=n-1
            #print(line)
    return n
            

In [14]:
def Nverb(s): #计算从句数量
    n=0
    #print(nlp.parse(s))
    parse=nlp.parse(s).split()
    #print(parse)
    for line in parse:
        #print(line)
        if line in ('(VBP','(VBZ','(VBD','(MD'):
            n+=1
            #print(line)
        #if line in ('(TO'):
        #    n=n-1
            #print(line)
    return n

In [15]:
def Ndpclause(s): #计算关系从句数量
    n=0
    #print(nlp.parse(s))
    parse=nlp.parse(s).split()
    #print(parse)
    for line in parse:
        #print(line)
        if line=='(SBAR':
            n+=1
            #print(line)
        #if line in ('(TO'):
        #    n=n-1
            #print(line)
    return n

In [16]:
s='I like coffee.'
print(nlp.parse(s))

(ROOT
  (S
    (NP (PRP I))
    (VP (VBP like)
      (NP (NN coffee)))
    (. .)))


In [17]:
def fragsT(s): #定位了各类括号，找到句子中的各种结构
    parse=nlp.parse(s) #.replace(' ','')
    parsepos=""
    frags=[]
    print(parse)
    cln=[]
    crn=[]
    x1=0
    #x2=0
    #y1=0
    y2=0
    x0=0
    for n1 in range(0,len(parse)):
        if parse[n1]=='(':
            t=0
            t0=0
            while n1-t0-1>=0 and parse[n1-1-t0]==' ':
                t0+=1
            if parse[n1-1-t0]=='\n' :
                t=t0
            else:
                t=0
            x1+=1
            y1=0
            for cha in parse[n1:]:
                if cha==")":
                    y1+=1
            cln.append(((n1,x1-1,y1),t))
        elif parse[n1]==")":
            x2=x1
            y2=0
            for cha in parse[n1:]:
                if cha==")":
                    y2+=1
            crn.append((n1,x1,y2-1))
    #print((cln,crn))
    #print(len(cln),len(crn))
    for cl in cln:
        frag=""
        n=0
        while frag=="" and n<=len(crn):
                cr=crn[n]
                if cl[0][0]<cr[0] and  cl[0][1]-cr[2]==cr[1]-cl[0][2]:
                    #print(((cl,cr)))
                    frag=parse[cl[0][0]:cr[0]+1]
                    frags.append((frag,cl[1]))
                n+=1
    result=[frags[0]]
    for n in range(1,len(frags)):
        n1=n
        while n1>=0 and frags[n1][1]==0:
            n1=n1-1
        result.append((frags[n][0],frags[n1][1]))
    return result
    
    

In [18]:
def match(lx,ly):
    if len(lx)>=len(ly):
        l1=lx
        l2=ly
    else:
        l1=ly
        l2=lx
    for n1 in range(0,len(l1)):
        if l1[n1]==l2[0]:
            if l1[n1:n1+len(l2)]==l2:
                return True
    return False

In [19]:
def frags(s): #定位了各类括号，找到句子中的各种结构 #还是不对
    parse=nlp.parse(s).replace('\r\n','') #.replace(' ','')
    parsepos=""
    frags=[]
    print(parse)
    cln=[]
    crn=[]
    x1=0
    #x2=0
    #y1=0
    y2=0
    x0=0
    for n1 in range(0,len(parse)):
        if parse[n1]=='(':
            x1+=1
            y1=0
            for cha in parse[n1:]:
                if cha==")":
                    y1+=1
            cln.append((n1,x1-1,y1))
        elif parse[n1]==")":
            x2=x1
            y2=0
            for cha in parse[n1:]:
                if cha==")":
                    y2+=1
            crn.append((n1,x1,y2-1))
    #print((cln,crn))
    #print(len(cln),len(crn))
    for cl in cln:
        frag=""
        n=0
        while frag=="" and n<=len(crn):
                cr=crn[n]
                if cl[0]<cr[0] and  cl[1]-cr[2]==cr[1]-cl[2]:
                    #print(((cl,cr)))
                    frag=parse[cl[0]:cr[0]+1]
                    frags.append(frag)
                n+=1
    return frags
    
    

In [20]:
def find_clause_component(s):
    
    target=frags(s)
    #print(target)
    result=[]
    for c in target:
        c=c.replace("("," ").replace(")",' ')
        c=c.split(' ')
        templ=[]
        for com in c:
            if com!='':
                templ.append(com)
        result.append(templ)
    return result
    

In [21]:
def find_clause_componentT(s):
    
    target=fragsT(s)
    print(target)
    result=[]
    for c in target:
        com=c[0].replace("("," ").replace(")",' ').replace('\r\n',"")
        com=com.split(' ')
        templ=[]
        for p in com:
            if p!='':
                templ.append((p,c[1]))
        result.append(templ)
    result1=[]
    templ1=[]
    #print(result) 
    for t in result[0]: #[('NN', 8), ('beauty', 8)]
        #for t in l: #('ROOT', 0)
            print(t[0]) 
            templ1.append(t[0])
    result1.append((templ1,(result[0][0][1],0)))
    for n in range(1,len(result)):
        templ=[]
        for c in result[n]: #result[n]是个由成分和从属组成的tuple的列表
            templ.append(c[0])
        n1=n-1
        #print('【check】',result[n][0][1],",",result[n1][0][1])
        while n1>=0 and result[n1][0][1]>=result[n][0][1]:
            #print('【check】',result[n][0][1],",",result[n1][0][1])
            n1=n1-1
        result1.append((templ,(result[n1][0][0],n1)))
    return result1
    

In [22]:
def find_np(s):
    target=frags(s)
    
    result=[]
    for c in target:
        print(c)
        c=c.replace("("," ").replace(")",' ')
        c=c.split(' ')
        if c!='':
            result.append(c)
        #print(c)
    rresult=[]
    for c in result:
        if c[1]=='NP':
            rresult.append(c)
    rrresult=[]
    for c1 in rresult:
        for c2 in rresult:
            if not match(c1,c2):
                rrresult.append(c1)
    return rresult

In [23]:
def find_np(s):
    target=frags(s)
    
    result=[]
    for c in target:
        if c[:3]=='(NP':
            result.append(c)
    return result


In [24]:
def find_cnp(s): #find complex np
    target=find_np(s)
    result=[]
    for c1 in target:
        n=0
        for c2 in target:
            if c1 in c2:
                n+=1
        if n<=1:
            for c2 in ("(PP",'(VBN','JJ','SBAR','(S(','(VBG('):
                if c2 in c1 and c1 not in result:
                    result.append(c1)
    return result

In [25]:
def Ncnp(s):
    target=find_cnp(s)
    return len(target)

In [26]:
def fdsyncomT2(s): #开始修改
    CN=[] #子句数量
    SBARS=[] #依存从句数量
    RS=[] #sum of root 句子数量
    VPS=[]
    TN=[] #T-unit 结构 
    CTN=[] #复杂T-unit 结构 
    target=find_clause_componentT(s) # 由[(['ADVP', 'RB', 'First'], ('FRAG', 1)) ,....]样式的元组构成的列表
    
    for n in range(0,len(target)):
        # target[n] 的样式 为 (['ADVP', 'RB', 'First'], ('FRAG', 1)) 
        if target[n][0][0]=='ROOT':
            RS.append(target[n])
            for n1 in range(n+1,len(target)):
                if target[n1][0][0]=='FRAG' and target[n1][1][1]==n:
                    TN.append(target[n1])  #统计T-UNIT FRAG>ROOT  root里有vp算Tunit
                    if 'VP' not in target[n][0]:
                        CN.append(target[n1])  #统计从句 FRAG>ROOT !<< VP 这里有个疑问 root里没vp算clause
                    
                    
            
        #统计从句  #“S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)”
        if target[n][0][0] in ('S','SINV','SQ'):
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0]=='VP':
                        #CN.append(target[n])
                        if target[n1][0][1] in ('VBD','VBP','VBZ','MD'):
                            CN.append(target[n])
        
        # 统计依存从句 SBAR<(S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)) 
        if target[n][0][0] =='SBAR':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SINV','SQ'):
                        for n2 in range(n1+1,len(target)):
                            if target[n2][1][1]==n1:
                                if target[n2][0][0]=='VP':
                                    #SBARS.append(target[n])
                                    if target[n2][0][1] in('VBD','VBP','VBZ','MD'):
                                        SBARS.append(target[n])
        
        # 统计T-UNIT : S|SBARQ|SINV|SQ>ROOT  直接从属于root
        if target[n][0][0]=='ROOT':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SBARQ','SINV','SQ'):
                        TN.append(target[n1])
                        # 统计 Complex T-UNIT S|SBARQ|SINV|SQ [> ROOT] << (SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)))
                        for n2 in range(1,len(target[n1][0])):
                            if target[n1][0][n2]=='SBAR':
                                tempn=1
                                n3=n2+tempn
                                while n3<=len(target[n1][0]) and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<=len(target[n1][0]):
                                    CTN.append(target[n1])
                                    
                            
        # 统计T-UNIT : S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP] 与 从句并列且不属于SBAR或VP
        if target[n][0][0] in ('S','SBARQ','SINV','SQ') and n-1>0:
            for n1 in range(0,n-1): #target[n1] 查看并列情况
                if target[n1][1]==target[n][1] and target[n1][0][0] in ('S','SBARQ','SINV','SQ'): 
                    tempn=0
                    for n2 in range(0,len(target)): #target[n2] 查看 是否从属于 从句结构
                        if tempn==0 and target[n2][0][0] in ('VP','SBAR'):
                            if len(target[n2][0])>len(target[n][0]) and match(target[n2][0],target[n][0]):
                        #如果前面计数是0 且 存在并列结构
                                tempn+=1
                    if tempn==0:
                        TN.append(target[n])
                    # 统计Complex T_Unit:"S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP]] <<(SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)"
                        for n3 in range(1,len(target[n][0])): #查看是否有复杂从句归属
                            if target[n][0][n3]=='SBAR': #是否有复杂从句归属
                                tempn=1
                                n4=n3+tempn
                                while n4<=len(target[n][0]) and target[n][0][n4] not in ('MD','VBP','VBZ','VBD'):
                                    n4+=1
                                if n4<=len(target[n][0]):
                                    CTN.append(target[n])
                        
                                        
                                        


############################################################至此完成        


    
    CNN=Ncnp(s)
    VN=Nverb(s)
    print('\n\n')
    print("*"*50,'COMPONENTS','*'*50)
    print('1. SENTENCE NUM:','\t',RS)
    print('2. CLAUSE NUM:','\t',CN)
    print('3. T_UNIT NUM:','\t',TN)
    print('4. DEPENDENT CLAUSE NUM:','\t',SBARS)
    print('5. COMPLEX T_UNIT NUM:','\t',CTN)
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    print('\n\n')
    print(s)
    print(nlp.parse(s))
    print('*'*50,'D-----A-----T-----A','*'*50)
    print('\n\n')
    print('1. SENTENCE NUM:','\t',len(RS))
    print('2. CLAUSE NUM:','\t',len(CN))
    print('3. T_UNIT NUM:','\t',len(TN))
    print('4. DEPENDENT CLAUSE NUM:','\t',len(SBARS))
    print('5. COMPLEX T_UNIT NUM:','\t',len(CTN))
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    return(len(RS),len(CN),len(TN),len(SBARS),len(CTN),CNN,VN)
            
            

In [27]:
def fdclausecom(s): 
    SS=0
    SBARS=0
    RS=0
    VPS=0
    CTN=0
    target=find_clause_component(s)
    for line in target:
        if line[1]=='ROOT':
            RS+=1
        if line[1]=='S' and len(line)<=2 or line[1]=='S'and line[2]!='S':
            SS+=1
            if 'SBAR' in line:
                CTN+=1
        if line[1]=='SBAR':
            SBARS+=1
        if line[1]=='VP':
            VPS+=1
    TN=SS-SBARS
    CN=SS
    CNN=Ncnp(s)
    VN=Nverb(s)
    print('SENTENCE NUM:',RS)
    print('CLAUSE NUM:',CN)
    print('T_UNIT NUM:',TN)
    print('DEPENDENT CLAUSE NUM:',SBARS)
    print('COMPLEX T_UNIT NUM:',CTN)
    print('COMPLEX NOUN PHRASE NUM:',CNN)
    print('VERB PHRASE NUM:',VN)
    return(RS,CN,TN,SBARS,CTN,CNN,VN)

In [28]:
def fdsyncomT1(s): #开始修改
    CN=0 #子句数量
    SBARS=0 #依存从句数量
    RS=0 #sum of root 句子数量
    VPS=0
    TN=0 #T-unit 结构 
    CTN=0 #复杂T-unit 结构 
    target=find_clause_componentT(s) # 由[(['ADVP', 'RB', 'First'], ('FRAG', 1)) ,....]样式的元组构成的列表
    
    for n in range(0,len(target)):
        # target[n] 的样式 为 (['ADVP', 'RB', 'First'], ('FRAG', 1)) 
        if target[n][0][0]=='ROOT':
            RS+=1
            for n1 in range(n+1,len(target)):
                if target[n1][0][0]=='FRAG' and target[n1][1][1]==n:
                    TN+=1  #统计T-UNIT FRAG>ROOT  root里有vp算Tunit
                    if 'VP' not in target[n][0]:
                        CN+=1  #统计从句 FRAG>ROOT !<< VP 这里有个疑问 root里没vp算clause
                    
                    
            
        #统计从句  #“S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)”
        if target[n][0][0] in ('S','SINV','SQ'):
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('VBD','VBP','VBZ'):
                        CN+=1
                    if target[n1][0][0]=='MD' and target[n1][0][1]=='VP':
                        CN+=1
        
        # 统计依存从句 SBAR<(S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)) 
        if target[n][0][0] =='SBAR':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SINV','SQ'):
                        for n2 in range(n1+1,len(target)):
                            if target[n2][1][1]==n1:
                                if target[n2][0][0] in ('VBD','VBP','VBZ'):
                                    SBARS+=1
                                if target[n2][0][:2]==('VP','MD'):
                                    SBARS+=1
        
        # 统计T-UNIT : S|SBARQ|SINV|SQ>ROOT  直接从属于root
        if target[n][0][0]=='ROOT':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SBARQ','SINV','SQ'):
                        TN+=1
                        # 统计 Complex T-UNIT S|SBARQ|SINV|SQ [> ROOT] << (SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)))
                        for n2 in range(1,len(target[n1][0])):
                            if target[n1][0][n2]=='SBAR':
                                tempn=1
                                n3=n2+tempn
                                while n3<=len(target[n1][0]) and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<=len(target[n1][0]):
                                    CTN+=1
                                    
                            
        # 统计T-UNIT : S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP] 与 从句并列且不属于SBAR或VP
        if target[n][0][0] in ('S','SBARQ','SINV','SQ') and n-1>0:
            for n1 in range(0,n-1): #target[n1] 查看并列情况
                if target[n1][1]==target[n][1] and target[n1][0][0] in ('S','SBARQ','SINV','SQ'): 
                    tempn=0
                    for n2 in range(0,len(target)): #target[n2] 查看 是否从属于 从句结构
                        if tempn==0 and target[n2][0][0] in ('VP','SBAR'):
                            if len(target[n2][0])>len(target[n][0]) and match(target[n2][0],target[n][0]):
                        #如果前面计数是0 且 存在并列结构
                                tempn+=1
                    if tempn==0:
                        TN+=1
                    # 统计Complex T_Unit:"S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP]] <<(SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)"
                        for n3 in range(1,len(target[n][0])): #查看是否有复杂从句归属
                            if target[n][0][n3]=='SBAR': #是否有复杂从句归属
                                tempn=1
                                n4=n3+tempn
                                while n4<=len(target[n][0]) and target[n][0][n4] not in ('MD','VBP','VBZ','VBD'):
                                    n4+=1
                                if n4<=len(target[n][0]):
                                    CTN+=1
                        
                                        
                                        


############################################################至此完成        


    
    CNN=Ncnp(s)
    VN=Nverb(s)
    
    print('\n\n')
    print(s)
    print(nlp.parse(s))
    print('*'*50,'D-----A-----T-----A','*'*50)
    print('\n\n')
    print('1. SENTENCE NUM:','\t',RS)
    print('2. CLAUSE NUM:','\t',CN)
    print('3. T_UNIT NUM:','\t',TN)
    print('4. DEPENDENT CLAUSE NUM:','\t',SBARS)
    print('5. COMPLEX T_UNIT NUM:','\t',CTN)
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    return(RS,CN,TN,SBARS,CTN,CNN,VN)
            
            

In [29]:
def fdsyncomT3(s): #暂时完成！！
    CN=[] #子句数量
    SBARS=[] #依存从句数量
    RS=[] #sum of root 句子数量
    VPS=[]
    TN=[] #T-unit 结构 
    CTN=[] #复杂T-unit 结构 
    target=find_clause_componentT(s) # 由[(['ADVP', 'RB', 'First'], ('FRAG', 1)) ,....]样式的元组构成的列表
    
    for n in range(0,len(target)):
        # target[n] 的样式 为 (['ADVP', 'RB', 'First'], ('FRAG', 1)) 
        if target[n][0][0]=='ROOT':
            RS.append(target[n])
            for n1 in range(n+1,len(target)-1):
                if target[n1][0][0]=='FRAG' and target[n1][1][1]==n:
                    if 'VP' in target[n][0]:
                        TN.append(target[n1])  #统计T-UNIT FRAG>ROOT  root里有vp算Tunit
                        for n2 in range(1,len(target[n1][0])): #查看是否有复杂从句归属
                            if target[n1][0][n2]=='SBAR': #是否有复杂从句归属
                                n3=n2+1
                                while n3<=len(target[n1][0])-1 and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<len(target[n1][0]):
                                    CTN.append(target[n1])
                    else:
                        CN.append(target[n1])  #统计从句 FRAG>ROOT !<< VP 这里有个疑问 root里没vp算clause
                    
                    
            
        #统计从句  #“S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)”
        if target[n][0][0] in ('S','SINV','SQ') and 'VP' in target[n][0][1:]:
            for n1 in range(n+1,len(target)):
                if target[n1][0][0]=='VP': 
                    if target[n1][1][1]==n: #VP直接从属于 S 
                        #CN.append(target[n])
                        n2=1
                        while n2<=len(target[n1][0])-1 and target[n1][0][n2] not in ('VBD','VBP','VBZ','MD'):
                            print('【check】',n2,target[n1],target[n1][0][n2])
                            n2+=1
                        if n2<len(target[n1][0]):  #保证VP下面有VBD,VBP,VBZ,MD 算法
                            if target[n] not in CN:
                                CN.append(target[n])

                    else:                  #VP间接从属于 S不能计算入内
                        for n3 in range(1,n1-1): 
                            if target[n1][1][1]==n3 and target[n3][1][1]==n:
                                n2=1
                                while n2<=len(target[n1][0])-1 and target[n1][0][n2] not in ('VBD','VBP','VBZ','MD'):
                                    n2+=1
                                if n2<len(target[n1][0]):  #保证VP下面有VBD,VBP,VBZ,MD 算法
                                    if target[n] not in CN:
                                        CN=CN
                                        #CN.append(target[n])
                                
        
        # 统计依存从句 SBAR<(S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)) 
        if target[n][0][0] =='SBAR':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SINV','SQ') and 'VP' in target[n1][0][1:]:      
                        for n2 in range(n1+1,len(target)):
                            if target[n2][0][0]=='VP': #VP直接从属于 S 
                                if target[n2][1][1]==n1:
                                    n3=1
                                    while n3<=len(target[n2][0])-1 and target[n2][0][n3] not in ('VBD','VBP','VBZ','MD'):
                                        n3+=1
                                    if n3<len(target[n2][0]):  #保证VP下面有VBD,VBP,VBZ,MD 算法
                                        if target[n] not in SBARS:
                                            SBARS.append(target[n])
                                else:                  #VP间接从属于 S
                                    for n4 in range(1,n2-1): 
                                        if target[n2][1][1]==n4 and target[n4][1][1]==n1:
                                            n5=1
                                            while n5<=len(target[n2][0])-1 and target[n2][0][n5] not in ('VBD','VBP','VBZ','MD'):
                                                n5+=1
                                            if n5<len(target[n2][0]):  #保证VP下面有VBD,VBP,VBZ,MD 算法
                                                if target[n] not in SBARS:
                                                    SBARS.append(target[n])
                                        
        # 统计T-UNIT : S|SBARQ|SINV|SQ>ROOT  直接从属于root
        if target[n][0][0]=='ROOT':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SBARQ','SINV','SQ'):
                        if target[n1] not in TN:
                            TN.append(target[n1])
                        # 统计 Complex T-UNIT S|SBARQ|SINV|SQ [> ROOT] << (SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)))
                        for n2 in range(1,len(target[n1][0])):
                            if target[n1][0][n2]=='SBAR':  #查看是否有复杂从句归属
                                n3=n2+1
                                while n3<=len(target[n1][0])-1 and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<len(target[n1][0]):
                                    if target[n1] not in CTN:
                                        CTN.append(target[n1])
                                    
                            
        # 统计T-UNIT : S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP] 与 从句并列且不属于SBAR或VP
        if target[n][0][0] in ('S','SBARQ','SINV','SQ') and n-1>0:
            for n1 in range(0,n-1): #target[n1] 查看并列情况
                if target[n1][1]==target[n][1] and target[n1][0][0] in ('S','SBARQ','SINV','SQ'): 
                    tempn=0
                    for n2 in range(0,len(target)-1): #target[n2] 查看 是否从属于 从句结构
                        if tempn==0 and target[n2][0][0] in ('VP','SBAR'):
                            if len(target[n2][0])>=len(target[n][0]) and match(target[n2][0],target[n][0]):
                        #如果前面计数是0 且 存在并列结构
                                tempn+=1
                    if tempn==0:
                        TN.append(target[n])
                    # 统计Complex T_Unit:"S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP]] <<(SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)"
                        for n3 in range(1,len(target[n][0])): #查看是否有复杂从句归属
                            if target[n][0][n3]=='SBAR': #是否有复杂从句归属
                                n4=n3+1
                                while n4<=len(target[n][0])-1 and target[n][0][n4] not in ('MD','VBP','VBZ','VBD'):
                                    n4+=1
                                if n4<len(target[n][0]):
                                    CTN.append(target[n])
                        
                                        
                                        


############################################################至此完成        


    
    CNN=Ncnp(s)
    VN=Nverb(s)
    print('\n\n')
    print("*"*50,'COMPONENTS','*'*50)
    print('1. SENTENCE NUM:','\t',RS)
    print('2. CLAUSE NUM:','\t',CN)
    print('3. T_UNIT NUM:','\t',TN)
    print('4. DEPENDENT CLAUSE NUM:','\t',SBARS)
    print('5. COMPLEX T_UNIT NUM:','\t',CTN)
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    print('\n\n')
    print(s)
    print(nlp.parse(s))
    print('*'*50,'D-----A-----T-----A','*'*50)
    print('\n\n')
    print('1. SENTENCE NUM:','\t',len(RS))
    print('2. CLAUSE NUM:','\t',len(CN))
    print('3. T_UNIT NUM:','\t',len(TN))
    print('4. DEPENDENT CLAUSE NUM:','\t',len(SBARS))
    print('5. COMPLEX T_UNIT NUM:','\t',len(CTN))
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    return(len(RS),len(CN),len(TN),len(SBARS),len(CTN),CNN,VN)
            

In [30]:
def fdsyncomT4(s): #完成！！
    CN=[] #子句数量
    SBARS=[] #依存从句数量
    RS=[] #sum of root 句子数量
    VPS=[]
    TN=[] #T-unit 结构 
    CTN=[] #复杂T-unit 结构 
    target=find_clause_componentT(s) # 由[(['ADVP', 'RB', 'First'], ('FRAG', 1)) ,....]样式的元组构成的列表
    
    for n in range(0,len(target)):
        # target[n] 的样式 为 (['ADVP', 'RB', 'First'], ('FRAG', 1)) 
        if target[n][0][0]=='ROOT':
            RS.append(target[n])
            for n1 in range(n+1,len(target)-1):
                if target[n1][0][0]=='FRAG' and target[n1][1][1]==n:
                    if 'VP' in target[n][0]:
                        TN.append(target[n1])  
                        #统计T-UNIT FRAG>ROOT  root里有vp算Tunit
                        for n2 in range(1,len(target[n1][0])): #查看是否有复杂从句归属
                            if target[n1][0][n2]=='SBAR': #是否有复杂从句归属
                                n3=n2+1
                                while n3<=len(target[n1][0])-1 and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<len(target[n1][0]):
                                    CTN.append(target[n1])
                    else:
                        CN.append(target[n1])  #统计从句 FRAG>ROOT !<< VP 这里有个疑问 root里没vp算clause
                    
        #统计从句  #“S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)”
        if target[n][0][0] in ('S','SINV','SQ') and 'VP' in target[n][0][1:]:
            for n1 in range(n+1,len(target)):
                if target[n1][0][0]=='VP' and target[n1][0][1] in ('VBD','VBP','VBZ','MD'):
                    if target[n1][1][1]==n: #VP直接从属于 S 
                        if target[n] not in CN:
                            CN.append(target[n])  #VP下面直接存在 VBD VBP VBZ 和 MD 

                    else:                  #VP间接从属于 S不能计算入内
                        for n3 in range(1,n1-1): 
                            if target[n1][1][1]==n3 and target[n3][1][1]==n:
                                n2=1
                                while n2<=len(target[n1][0])-1 and target[n1][0][n2] not in ('VBD','VBP','VBZ','MD'):
                                    n2+=1
                                if n2<len(target[n1][0]):  #保证VP下面有VBD,VBP,VBZ,MD 算法
                                    if target[n] not in CN:
                                        CN=CN
                                        #CN.append(target[n])
                                
        
        # 统计依存从句 SBAR<(S|SINV|SQ<(VP<#MD|VBD|VBP|VBZ)) 
        if target[n][0][0] =='SBAR':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SINV','SQ') and 'VP' in target[n1][0][1:]:      
                        for n2 in range(n1+1,len(target)):
                            if target[n2][0][0]=='VP' and target[n2][0][1] in ('VBD','VBP','VBZ','MD'): #VP直接从属于 S 
                                    if target[n] not in SBARS:
                                        SBARS.append(target[n])
                                #不考虑VP间接从属于 S

                                        
        # 统计T-UNIT : S|SBARQ|SINV|SQ>ROOT  直接从属于root
        if target[n][0][0]=='ROOT':
            for n1 in range(n+1,len(target)):
                if target[n1][1][1]==n: 
                    if target[n1][0][0] in ('S','SBARQ','SINV','SQ'):
                        if target[n1] not in TN:
                            TN.append(target[n1])
                        # 统计 Complex T-UNIT S|SBARQ|SINV|SQ [> ROOT] << (SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)))
                        for n2 in range(1,len(target[n1][0])):
                            if target[n1][0][n2]=='SBAR':  #查看是否有复杂从句归属
                                n3=n2+1
                                while n3<=len(target[n1][0])-1 and target[n1][0][n3] not in ('MD','VBP','VBZ','VBD'):
                                    n3+=1
                                if n3<len(target[n1][0]):
                                    if target[n1] not in CTN:
                                        CTN.append(target[n1])
                                    
                            
        # 统计T-UNIT : S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP] 与 从句并列且不属于SBAR或VP
        if target[n][0][0] in ('S','SBARQ','SINV','SQ') and n-1>0:
            for n1 in range(0,n-1): #target[n1] 查看并列情况
                if target[n1][1]==target[n][1] and target[n1][0][0] in ('S','SBARQ','SINV','SQ'): 
                    tempn=0
                    for n2 in range(0,len(target)-1): #target[n2] 查看 是否从属于 从句结构
                        if tempn==0 and target[n2][0][0] in ('VP','SBAR'):
                            if len(target[n2][0])>=len(target[n][0]) and match(target[n2][0],target[n][0]):
                        #如果前面计数是0 且 存在并列结构
                                tempn+=1
                    if tempn==0:
                        TN.append(target[n])
                    # 统计Complex T_Unit:"S|SBARQ|SINV|SQ[$-- S|SBARQ|SINV|SQ !>> SBAR|VP]] <<(SBAR<(S|SQ|SINV<(VP<#MD|VBP|VBZ|VBD)"
                        for n3 in range(1,len(target[n][0])): #查看是否有复杂从句归属
                            if target[n][0][n3]=='SBAR': #是否有复杂从句归属
                                n4=n3+1
                                while n4<=len(target[n][0])-1 and target[n][0][n4] not in ('MD','VBP','VBZ','VBD'):
                                    n4+=1
                                if n4<len(target[n][0]):
                                    CTN.append(target[n])
                        
                                        
                                        


############################################################至此完成        


    
    CNN=Ncnp(s)
    VN=Nverb(s)
    print('\n\n')
    print("*"*50,'COMPONENTS','*'*50)
    print('1. SENTENCE NUM:','\t',RS)
    print('2. CLAUSE NUM:','\t',CN)
    print('3. T_UNIT NUM:','\t',TN)
    print('4. DEPENDENT CLAUSE NUM:','\t',SBARS)
    print('5. COMPLEX T_UNIT NUM:','\t',CTN)
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    print('\n\n')
    print(s)
    print(nlp.parse(s))
    print('*'*50,'D-----A-----T-----A','*'*50)
    print('\n\n')
    print('1. SENTENCE NUM:','\t',len(RS))
    print('2. CLAUSE NUM:','\t',len(CN))
    print('3. T_UNIT NUM:','\t',len(TN))
    print('4. DEPENDENT CLAUSE NUM:','\t',len(SBARS))
    print('5. COMPLEX T_UNIT NUM:','\t',len(CTN))
    print('6. COMPLEX NOUN PHRASE NUM:','\t',CNN)
    print('7. VERB PHRASE NUM:','\t',VN)
    return(len(RS),len(CN),len(TN),len(SBARS),len(CTN),CNN,VN)
            

In [31]:
def readsent(file):
    list_sents=[]
    
    text=open(file,encoding='utf-8',errors='ignore')
        
    for line in text:
        line=line.strip()
        sent=nltk.sent_tokenize(line)
        
        list_sents+=sent
    return list_sents

In [32]:
def SenComAnalyser3(file):
    target=readsent(file)
    data=[]
    sumWN=[]
    result=''
    datatitle=['W','S','C','T','DC','CT','CN','VP']
    headline='SENT\COMP'
    for t in datatitle:
        headline+='\t'+t
    result+=headline+'\n'
    print(headline)
    n=1
    for line in target:
        WN=len(nltk.word_tokenize(line))
        l=''
        l+='sentence'+str(n)+'\t'
        l+=str(WN)+'\t'
        for d in fdclausecom(line):
            l+=str(d)+'\t'
        result+=l+'\n'
        n+=1
        #print('sentence'+str(n),'\t',l)
    print(result)

In [33]:
def SenComAnalyser4(file):
    target=readsent(file)
    data=[]
    sumWN=[]
    result=''
    datatitle=['W','S','C','T','DC','CT','CN','VP']
    headline='SENT\COMP'
    for t in datatitle:
        headline+='\t'+t
    result+=headline+'\n'
    print(headline)
    n=1
    for line in target:
        WN=len(nltk.word_tokenize(line))
        l=''
        l+='sentence'+str(n)+'\t'
        l+=str(WN)+'\t'
        for d in fdsyncomT3(line):
            l+=str(d)+'\t'
        result+=l+'\n'
        n+=1
        #print('sentence'+str(n),'\t',l)
    print(result)

In [35]:
#SenComAnalyser3(file)

In [36]:
def SenComAnalyser1(file):
    target=readsent(file)
    data=[]
    sumWN=[]
    for line in target:
        WN=len(nltk.word_tokenize(line))
        sumWN.append(WN)
        data.append(fdclausecom(line))
    
    S=''
    T=''
    C=""
    CT=""
    DC=""
    CN=""
    VP=""
    datas=[S,C,T,DC,CT,CN,VP]
    datatitle=['W','S','C','T','DC','CT','CN','VP']
    n=0
    sumresult=[]
    W=''
    for c in sumWN:
         W+=str(c)+'\t'
    sumresult.append(W)
    for l in datas:
        for c in data:
            #print(c)
            l+=str(c[n])+'\t'
        #print(l)
        sumresult.append(l)
        n+=1
    headline='components'
    for n in range(1,len(target)+1):
        headline+='\t'+'sentence'+str(n)
    print(headline)
    for n in range(0,len(sumresult)):
        print(datatitle[n],'\t',sumresult[n])
       
        

In [37]:
def SenComAnalyser2(file):
    target=readsent(file)
    data=[]
    sumWN=[]
    result=''
    datatitle=["W","W/T","W/C","C","C/T","CT/T","DC/C","DC/T","T",'CN',"CN/C","CN/T","VP/C"]
    headline='\t长度特征 \t \t \t子句数量 \t从属情况 \t \t \t \t并列情况 \t特殊结构占比\t \t \t'
    result+=headline+'\n'
    headline='SENT\COMP'
    for t in datatitle:
        headline+='\t'+t
    result+=headline+'\n'
    print(headline)
    n=1
    
        
    for line in target:
        w=len(nltk.word_tokenize(line))
        l=''
        l+='sentence'+str(n)+'\t'
        l+=str(w)+'\t'   #W
        data=fdclausecom(line)
        if data[2] !=0 :       #W/T
            l+=str(round(w/data[2],2))  +'\t' 
        else:
            l+='0'+'\t'
        if data[1] !=0 :  #W/C
            l+=str(round(w/data[1],2))  +'\t'
        else:
            l+='0'+'\t'
        l+=str(data[1])+'\t'   #C

        if data[2] !=0 :
            l+= str(round(data[1]/data[2],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[4]/data[2],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[1] !=0 :
            l+= str(round(data[3]/data[1],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[3]/data[2],2))   +'\t'
        else:
            l+='0'+'\t'
        l+=str(data[2])+'\t'
        l+=str(data[5])+'\t'
        if data[1] !=0 :
            l+= str(round(data[5]/data[1],2)) +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[5]/data[2],2)) +'\t'
        else:
            l+='0'+'\t'
        if data[1] !=0 :
            l+= str(round(data[6]/data[1],2)) +'\t'
        else:
            l+='0'+'\t'
        result+=l+'\n'
        n+=1
        #print('sentence'+str(n),'\t',l)
    print(result)

    
        

In [38]:
#fdsyncomT4(file)

In [39]:
def SenComAnalyser5(file):
    target=readsent(file)
    data=[]
    sumWN=[]
    result=''
    datatitle=["W","W/T","W/C","C","C/T","CT/T","DC/C","DC/T","T",'CN',"CN/C","CN/T","VP/C"]
    headline='\t长度特征 \t \t \t子句数量 \t从属情况 \t \t \t \t并列情况 \t特殊结构占比\t \t \t'
    result+=headline+'\n'
    headline='SENT\COMP'
    for t in datatitle:
        headline+='\t'+t
    result+=headline+'\n'
    print(headline)
    n=1
    W=0
    T=0
    C=0
    DC=0
    CT=0
    CN=0
    VP=0
    for line in target:
        w=len(nltk.word_tokenize(line))
        l=''
        l+='sentence'+str(n)+'\t'
        l+=str(w)+'\t'   #W
        W+=w
        data=fdsyncomT4(line)
        C+=data[1]
        T+=data[2]
        DC+=data[3]
        CT+=data[4]
        CN+=data[5]
        VP+=data[6]
        if data[2] !=0 :       #W/T
            l+=str(round(w/data[2],2))  +'\t' 
        else:
            l+='0'+'\t'
        if data[1] !=0 :  #W/C
            l+=str(round(w/data[1],2))  +'\t'
        else:
            l+='0'+'\t'
        l+=str(data[1])+'\t'   #C
        
        if data[2] !=0 :
            l+= str(round(data[1]/data[2],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[4]/data[2],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[1] !=0 :
            l+= str(round(data[3]/data[1],2))    +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[3]/data[2],2))   +'\t'
        else:
            l+='0'+'\t'
        l+=str(data[2])+'\t'
        l+=str(data[5])+'\t'
        if data[1] !=0 :
            l+= str(round(data[5]/data[1],2)) +'\t'
        else:
            l+='0'+'\t'
        if data[2] !=0 :
            l+= str(round(data[5]/data[2],2)) +'\t'
        else:
            l+='0'+'\t'
        if data[1] !=0 :
            l+= str(round(data[6]/data[1],2)) +'\t'
        else:
            l+='0'+'\t'
        result+=l+'\n'
        n+=1
        #print('sentence'+str(n),'\t',l)
    print(result)
    dataresult=(W,round(W/T,2),round(W/C,2),round(C/T,2),round(CT/T,2),round(DC/C,2),round(DC/T,2),T,CN,round(CN/C,2),round(CN/T,2),round(VP/C,2))
    return((W,T,C,DC,CT,CN,VP),(dataresult))
    
        

In [40]:
#fdsyncomT3(readsent(file)[6])

In [41]:
#file=r'C:\Users\spike\Documents\论文相关\Stanford NLP句法教学\b1u1.txt'
#SenComAnalyser5(file)

In [42]:
#from nltk.tree import Tree
#s=readsent(file)[6]
#s=text
#deparse= nlp.parse(s)
#tree=Tree.fromstring(deparse)
#tree.draw()

In [43]:
def stanz_text(txt): #将段落转换为数据表
    output=[]
    txt=nltk.sent_tokenize(txt)
    for sent in txt:
        output.append(stanz(sent))
    prtline=''
    text=''
    for line in output:
        #prtline+="\n"
        for item in line:
            if item[0] in string.punctuation:
                text+=item[0]
            else:
                text+=' '+item[0]
            for inf in item:
                prtline+=str(inf)+'\t'
            prtline+='\n'
        prtline+=50*'-'+'\n'
    print(prtline)
    print(text[1:])
    return output

In [44]:
#s = 'Unable to resist the impulse of curiosity, she raised her eyes to his.' #stanford NLP 示范
#doc=xnlp(s)
#print (lemma(s))
#print(gonver(s))
#print(*[f"index: {word.index.rjust(2)}\tword: {word.text.ljust(11)}\tgovernor index: {word.governor}\tgovernor: {(doc.sentences[0].words[word.governor-1].text if word.governor > 0 else 'root').ljust(11)}\tdeprel: {word.dependency_relation}" for word in doc.sentences[0].words], sep='\n')

In [46]:
s1='In a study of 33 years of trends in Body Mass Index  across 200 countries, the scientists found that people worldwide are getting heavier and that most of the rise is due to gains in BMI in rural areas.'

In [47]:
s2='In 1985, urban men and women in more than three quarters of the countries studied had higher BMIs than men and women in rural areas.'

In [73]:
from nltk.tree import Tree
deparse= nlp.parse(s)
tree=Tree.fromstring(deparse)
tree.draw()

In [64]:
from nltk.tree import Tree #将句子改为名词短语、形容词短语填图练习
#s=text
outcome=""
deparse= nlp.parse(s)
data=deparse.split()
print(data)
num=0
for n in range(0,len(deparse)):
    if n<=len(deparse)-2 and deparse[n]+deparse[n+1]=="NP" or n-1>=0 and deparse[n-1]+deparse[n]=="NP" or\
        n-3>=0 and deparse[n-3]+deparse[n-2]+deparse[n-1]+deparse[n] in ('ADJP',"ADVP") or\
        n-2>=0 and n<=len(deparse)-2 and deparse[n-2]+deparse[n-1]+deparse[n]+deparse[n+1] in ('ADJP',"ADVP") or\
        n-1>=0 and n<=len(deparse)-3 and deparse[n-1]+deparse[n]+deparse[n+1]+deparse[n+2] in ('ADJP',"ADVP") or\
        n<=len(deparse)-4 and deparse[n]+deparse[n+1]+deparse[n+2]+deparse[n+3] in ('ADJP',"ADVP"):
        outcome+=" "
    else:
        outcome+=deparse[n]
outcome2=''
no=1
for n in range(0,len(outcome)):
    if n-1>=0 and outcome[n-1]=="(" and outcome[n]==' ':
        outcome2+=str(no)+'____'
        no+=1
    else:
        outcome2+=outcome[n]
tree=Tree.fromstring(outcome2)
tree.draw()

['(ROOT', '(S', '(S', '(VP', '(VBG', 'Going)', '(PP', '(PP', '(IN', 'from)', '(NP', '(JJ', 'junior)', '(JJ', 'high)', '(NN', 'school)))', '(PP', '(TO', 'to)', '(NP', '(JJ', 'senior)', '(JJ', 'high)', '(NN', 'school))))))', '(VP', '(VBZ', 'is)', '(NP', '(DT', 'a)', '(RB', 'really)', '(JJ', 'big)', '(NN', 'challenge)))', '(.', '.)))']


In [85]:
from nltk.tree import Tree #将句子改为短语填图练习 此处改编针对性填空练习
s='Going from junior high school to senior high school is a really big challenge.'
outcome=""
deparse= nlp.parse(s)
deparse=deparse.replace('\r',"#").replace('\n',"@")
data=deparse.split()
print(data)
num=0
for n in range(0,len(data)):
    if data[n]=='(NP' and data[n+1]!='(TO': # 
        outcome+="( "
    elif data[n] in("(ADJP","(ADVP",'(NP','(VP','(PP',"(ADJP#@","(ADVP#@",'(NP#@','(VP#@','(PP#@'):
        outcome+=data[n]+' '
    elif data[n]=='(ROOT#@':
        outcome+='(Sentence#@'
    elif ")" not in data[n]:
        outcome+="("
    else:
        outcome+=data[n]+' '
        
print(outcome)

outcome=outcome.replace('#','\r').replace('@','\n')
print(outcome)
outcome2=''
no=1
for n in range(0,len(outcome)):
    if n-1>=0 and outcome[n-1]=="(" and outcome[n]==' ':
        outcome2+=str(no)+'____'
        no+=1
    else:
        outcome2+=outcome[n]
tree=Tree.fromstring(outcome2)
tree.draw()

['(ROOT#@', '(S#@', '(S#@', '(VP', '(VBG', 'Going)#@', '(PP#@', '(PP', '(IN', 'from)#@', '(NP', '(JJ', 'junior)', '(JJ', 'high)', '(NN', 'school)))#@', '(PP', '(TO', 'to)#@', '(NP', '(JJ', 'senior)', '(JJ', 'high)', '(NN', 'school))))))#@', '(VP', '(VBZ', 'is)#@', '(NP', '(DT', 'a)', '(RB', 'really)', '(JJ', 'big)', '(NN', 'challenge)))#@', '(.', '.)))']
(Sentence#@(((VP (Going)#@ (PP#@ (PP (from)#@ ( (junior) (high) (school)))#@ (PP (to)#@ ( (senior) (high) (school))))))#@ (VP (is)#@ ( (a) (really) (big) (challenge)))#@ (.))) 
(Sentence
(((VP (Going)
 (PP
 (PP (from)
 ( (junior) (high) (school)))
 (PP (to)
 ( (senior) (high) (school))))))
 (VP (is)
 ( (a) (really) (big) (challenge)))
 (.))) 


In [88]:
from nltk.tree import Tree #将句子改为短语图形
s='Going from junior high school to senior high school is a really big challenge.'
outcome=""
deparse= nlp.parse(s)
deparse=deparse.replace('\r',"#").replace('\n',"@")
data=deparse.split()
print(data)
num=0
for n in range(0,len(data)):
    #if data[n]=='(SBAR#@':
    #    outcome+="( "
    #elif data[n]=='(VP' and data[n+1]!='(TO': # 
    #    outcome+="( "
    if data[n] in("(ADJP","(ADVP",'(NP','(VP','(PP',"(ADJP#@","(ADVP#@",'(NP#@','(VP#@','(PP#@'):
        outcome+=data[n]+' '
    elif data[n]=='(ROOT#@':
        outcome+='(Sentence#@'
    elif ")" not in data[n]:
        outcome+="("
    else:
        outcome+=data[n]+' '
        
print(outcome)

outcome=outcome.replace('#','\r').replace('@','\n')
print(outcome)
outcome2=''
no=1
for n in range(0,len(outcome)):
    if n-1>=0 and outcome[n-1]=="(" and outcome[n]==' ':
        outcome2+=str(no)+'____'
        no+=1
    else:
        outcome2+=outcome[n]
tree=Tree.fromstring(outcome2)
tree.pretty_print()
tree.draw()

['(ROOT#@', '(S#@', '(S#@', '(VP', '(VBG', 'Going)#@', '(PP#@', '(PP', '(IN', 'from)#@', '(NP', '(JJ', 'junior)', '(JJ', 'high)', '(NN', 'school)))#@', '(PP', '(TO', 'to)#@', '(NP', '(JJ', 'senior)', '(JJ', 'high)', '(NN', 'school))))))#@', '(VP', '(VBZ', 'is)#@', '(NP', '(DT', 'a)', '(RB', 'really)', '(JJ', 'big)', '(NN', 'challenge)))#@', '(.', '.)))']
(Sentence#@(((VP (Going)#@ (PP#@ (PP (from)#@ (NP (junior) (high) (school)))#@ (PP (to)#@ (NP (senior) (high) (school))))))#@ (VP (is)#@ (NP (a) (really) (big) (challenge)))#@ (.))) 
(Sentence
(((VP (Going)
 (PP
 (PP (from)
 (NP (junior) (high) (school)))
 (PP (to)
 (NP (senior) (high) (school))))))
 (VP (is)
 (NP (a) (really) (big) (challenge)))
 (.))) 
                                                 Sentence                                        
                                                    |                                             
                                                                                         

In [126]:
print(nlp.parse(readsent(file)[6]))

(ROOT
  (S
    (S
      (NP (PRP I))
      (VP (VBP know)
        (SBAR
          (S
            (NP (IN that) (JJ Chinese))
            (VP (VBZ is)
              (NP (DT a)
                (ADJP (RB very) (JJ difficult))
                (NN language)))))))
    (, ,)
    (CC but)
    (S
      (NP (PRP I))
      (VP (VBP hope)
        (S
          (VP (TO to)
            (VP (VB be)
              (ADJP (JJ fluent))
              (SBAR
                (WHADVP (WRB when))
                (S
                  (NP (PRP I))
                  (VP (VBP graduate)))))))))
    (. .)))


In [157]:
history

import nltk
import string
import os
import stanfordnlp

from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'C:\Users\spike\python\stanford-corenlp-full-2018-10-05',lang='en')

xnlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos,lemma')

xnlp = stanfordnlp.Pipeline()
avoidance=string.punctuation+string.whitespace
def readfile(file):
    list_sents={}
    name=os.path.basename(file)
    if file[-4:]==".txt":
        text=open(file,encoding='utf-8',errors='ignore')
        avoidance=string.punctuation+string.whitespace
        option=("A.","B.","C.","D.","E.","F.","G.","B ","C ","D ","E ","F ","G ")
    #print(avoidance)
        line=''
        for chline in text:
            #for w in option:
                #chline=chline.replace(w," ")           
            for character in chline:
                #if character in option and chline[chline.index(character)+1] in ("."," "):
                    #line+=" "
                if character=="'" or character=='"' or charac